In [104]:
import pandas as pd
import plotly.graph_objects as go
import openpyxl as xl

In [9]:
import requests
import time
import pandas as pd
import numpy as np

def Convert(dfs,stock):
    names = []
    names_values = []
    basic_info = []
    try:
        df = dfs[5]
        basic_df = dfs[4]
        for i in range(0,len(df.columns)):
            if i%2 == 0:
                #display(df[i])
                names.append(df[i])
            else:
                names_values.append(df[i])
                
        Index_names = pd.concat(names).reset_index(drop= True)
        Index_Values = pd.concat(names_values).reset_index(drop= True)
        df1 = pd.DataFrame()
        df1[f'{stock}'] = Index_names
        df1['Values'] = Index_Values
        #df2 = df1['Values'].replace('%','')
        df2 = df1['Values'].replace('-','0')
        #df1 = df1.fillna('-1')
        
        #Adding stock basic info
        basic_df = basic_df.T.drop(0,axis = 1)
        basic_df = basic_df.rename({1:'Company',2:'Sector'},axis=1)
        basic_df.index = [stock]

        #Adds Percentages to name
        not_add_percentage = df1[~df1['Values'].str.contains('%')]
        not_add_percentage  = not_add_percentage[f'{stock}']
        add_percentage = df1[df1['Values'].str.contains('%')]
        add_percentage  = add_percentage[f'{stock}'] + ' %'

        #Converts that column so I can remove %
        df1['Values'] = df1['Values'].convert_dtypes(str)
        df1['Values'] = df1['Values'].str.replace('%','')

        df1['Values'] = df1['Values'].str.replace('K','e3')
        df1['Values'] = df1['Values'].str.replace('M','e6')
        df1['Values'] = df1['Values'].str.replace('B','e9')

        df1['Index'] = pd.concat([not_add_percentage,add_percentage]).sort_index()
        #df1.index = df1['Index']
        print(df1.index)
        df1 = df1.drop([f'{stock}','Index'], axis=1)
        print(df1)
        df1.columns = [f'{stock}']
        print(df1.columns)
        df2  = df1.T
        print(df2.index)
    
        #display(df2)
        for i in df2.columns: #converts everything to float
            try:
                df2['{}'.format(i)] = df2['{}'.format(i)].astype(float)
            except:
                #print(i)
                0
        df3 =  basic_df.join(df2, how='outer')
        #display(df2.T,basic_df,df3)
        
        return df3
    except:
        print(f'2-{stock}')
        
  
saved = []
stocks = pd.read_csv(r'C:\Users\Tafimul\Python Main Projects\Dashboard\Stock_Screener\SAVE\ETFs.csv')
print(len(stocks))
stocks = stocks.drop_duplicates('Ticker')
print(len(stocks))
#stocks = pd.read_csv('SAVE\ETFs.csv')
start = time.time()
for stock in stocks['Ticker'][0:2]:
  try:
    url = f'https://finviz.com/quote.ashx?t={stock}'

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    dfs = pd.read_html(r.text)
    #print(dfs)
    d = Convert(dfs,stock)
    #saved.append(d)
  except:
    print('1')
print(saved) 
#savethisdf = pd.concat(saved)
savethisdf.index.names = ['Tickers']
savethisdf[['Sector','Industry','Country']] = savethisdf['Sector'].str.split('|',expand=True)
savethisdf = savethisdf[savethisdf.columns[0:74]]
savethisdf = savethisdf.replace('-',np.nan)
savethisdf.to_csv('Dashboard\Stock_Screener\SAVE\FUND.csv')
end = time.time()
print(end - start)
  #return saved
#getFinData()

671
671
RangeIndex(start=0, stop=72, step=1)
            AA     Values         Index
0        Index          -         Index
1   Market Cap     6.49e9    Market Cap
2       Income   963.00e6        Income
3        Sales    13.39e9         Sales
4      Book/sh      31.13       Book/sh
..         ...        ...           ...
67         ATR       2.93           ATR
68  Volatility  7.00 5.88  Volatility %
69  Prev Close      37.52    Prev Close
70       Price      35.51         Price
71      Change      -5.36      Change %

[72 rows x 3 columns]
2-AA
RangeIndex(start=0, stop=72, step=1)
           AAL      Values         Index
0        Index     S&P 500         Index
1   Market Cap      8.23e9    Market Cap
2       Income  -1922.00e6        Income
3        Sales     40.72e9         Sales
4      Book/sh      -12.95       Book/sh
..         ...         ...           ...
67         ATR        0.63           ATR
68  Volatility   5.40 4.18  Volatility %
69  Prev Close       12.71    Prev Close


ValueError: Columns must be same length as key

In [5]:
saved

671

In [105]:
wb = xl.load_workbook('Networth.xlsx',data_only=True) # data only so it doesnt read the formulas
#print(wb.sheetnames)
ws = wb['Summary Table'] #This is the only thing we care about 
df = pd.DataFrame(ws.values) #converts it into a df 

#Organizes the dataframe
df = df.set_index(df[0])
df = df[df.columns[2:]].T
df = df.set_index('Date')

In [108]:
fig = go.Figure()
df1 = df.drop(columns= ['Assets','Liabilities'],axis = 0) # Drops the columns you don't need for the graph

#To make the bars 
for col in df1[:-1].columns:
    if col == 'Net Worth': 
        fig.add_trace(go.Scatter(x = df.index,y = df[f'{col}'], name = col)) # Adds the Networth line
    else:
        fig.add_bar(x = df.index,y = df[f'{col}'], name = col) 
        
#Updates the figure 
fig.update_xaxes(title = 'Time') 
fig.update_yaxes(title = 'Cash') 
fig.update_layout(barmode='relative', title_text='Networth Tracker')
print(df)

0           Assets Cash / Cash Equivalents Investments Retirement Investments  \
Date                                                                            
2022-01-01  372681                    4700       57939                   4942   
2022-02-01  359736                    1713     43546.4                5176.26   
2022-03-01  361090                    1500       47882                5707.97   
2022-04-01  380369                    4259       54675                 6434.7   
2022-05-01  383231                    4500     45027.3                6703.22   
2022-06-01  380718                    9935     32649.5                7524.86   
2022-07-01  417595                   24368     22667.8                 7558.8   
2022-08-01       0                       0           0                      0   
2022-09-01       0                       0           0                      0   
2022-10-01       0                       0           0                      0   
2022-11-01       0          

In [60]:
import talib as ta
import pandas as pd
df = pd.read_csv('Stock_Screener\SAVE\STOCKS_Daily.csv',index_col = 0,header=[0, 1])
Adj_Close = df['Adj Close'].astype(np.float)
Adj_Close['Date'] = pd.to_datetime(Adj_Close.index) - pd.to_timedelta(7, unit='d')
Adj_Close_Weekly = Adj_Close.groupby([pd.Grouper(key='Date', freq='W-FRI')]).mean()
save1 = []
save2 = []
save3 = []
for i in Adj_Close_Weekly[0:5]:
    #print(i)
    print(Adj_Close[i])
    #macd, macdsignal, macdhist = ta.MACD(Adj_Close[f'{i}'], fastperiod=12, slowperiod=26, signalperiod=9)
    real = ta.EMA(Adj_Close[f'{i}'], timeperiod=30)
    save1.append(real)
    #save1.append(macd)
    #save2.append(macdsignal)
    #save3.append(macdhist)
save1

Date
2020-10-02          NaN
2020-10-05    11.846516
2020-10-06    11.588335
2020-10-07    12.154346
2020-10-08    12.640916
                ...    
2022-09-26    34.470001
2022-09-27    34.700001
2022-09-28    35.230000
2022-09-29    35.430000
2022-09-30    33.660000
Name: AA, Length: 503, dtype: float64
Date
2020-10-02      NaN
2020-10-05    13.12
2020-10-06    12.53
2020-10-07    13.07
2020-10-08    13.16
              ...  
2022-09-26    11.86
2022-09-27    12.27
2022-09-28    12.75
2022-09-29    12.25
2022-09-30    12.04
Name: AAL, Length: 503, dtype: float64
Date
2020-10-02           NaN
2020-10-05    115.110458
2020-10-06    111.810295
2020-10-07    113.707390
2020-10-08    113.598701
                 ...    
2022-09-26    150.770004
2022-09-27    151.759995
2022-09-28    149.839996
2022-09-29    142.479996
2022-09-30    138.199997
Name: AAPL, Length: 503, dtype: float64
Date
2020-10-02           NaN
2020-10-05     80.484726
2020-10-06     78.617470
2020-10-07     79.697556
2020

[Date
 2020-10-02          NaN
 2020-10-05          NaN
 2020-10-06          NaN
 2020-10-07          NaN
 2020-10-08          NaN
                 ...    
 2022-09-26    45.886076
 2022-09-27    45.164394
 2022-09-28    44.523465
 2022-09-29    43.936790
 2022-09-30    43.273771
 Length: 503, dtype: float64,
 Date
 2020-10-02          NaN
 2020-10-05          NaN
 2020-10-06          NaN
 2020-10-07          NaN
 2020-10-08          NaN
                 ...    
 2022-09-26    13.659365
 2022-09-27    13.569728
 2022-09-28    13.516842
 2022-09-29    13.435111
 2022-09-30    13.345104
 Length: 503, dtype: float64,
 Date
 2020-10-02           NaN
 2020-10-05           NaN
 2020-10-06           NaN
 2020-10-07           NaN
 2020-10-08           NaN
                  ...    
 2022-09-26    156.881890
 2022-09-27    156.551445
 2022-09-28    156.118448
 2022-09-29    155.238548
 2022-09-30    154.139287
 Length: 503, dtype: float64,
 Date
 2020-10-02           NaN
 2020-10-05           Na

In [43]:
save1

[Date
 2015-06-15   NaN
 2015-06-22   NaN
 2015-06-24   NaN
 2015-06-25   NaN
 2015-06-26   NaN
               ..
 2022-09-23   NaN
 2022-09-26   NaN
 2022-09-28   NaN
 2022-09-29   NaN
 2022-09-30   NaN
 Length: 1679, dtype: float64,
 Date
 2015-06-15   NaN
 2015-06-22   NaN
 2015-06-24   NaN
 2015-06-25   NaN
 2015-06-26   NaN
               ..
 2022-09-23   NaN
 2022-09-26   NaN
 2022-09-28   NaN
 2022-09-29   NaN
 2022-09-30   NaN
 Length: 1679, dtype: float64,
 Date
 2015-06-15   NaN
 2015-06-22   NaN
 2015-06-24   NaN
 2015-06-25   NaN
 2015-06-26   NaN
               ..
 2022-09-23   NaN
 2022-09-26   NaN
 2022-09-28   NaN
 2022-09-29   NaN
 2022-09-30   NaN
 Length: 1679, dtype: float64,
 Date
 2015-06-15   NaN
 2015-06-22   NaN
 2015-06-24   NaN
 2015-06-25   NaN
 2015-06-26   NaN
               ..
 2022-09-23   NaN
 2022-09-26   NaN
 2022-09-28   NaN
 2022-09-29   NaN
 2022-09-30   NaN
 Length: 1679, dtype: float64,
 Date
 2015-06-15   NaN
 2015-06-22   NaN
 2015-06-24   NaN


In [70]:
import numpy as np
#real = ta.EMA(data = Adj_Close,dtype=np.float64, timeperiod=30) 
Adj_Close_Weekly.apply(lambda c: ta.MA(c,timeperiod=30))


,AA,AAL,AAPL,ABBV,ABC,ABNB,ABST,ACCD,ACHR,ACIW,...,YOU,Z,ZBRA,ZEN,ZG,ZI,ZM,ZS,ZUO,ZY
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-26,63.713106,15.897267,157.236590,148.409657,148.000839,131.545334,8.837461,11.525950,3.805900,28.266600,...,25.716650,42.315583,358.422883,95.823800,41.850450,46.359233,109.077383,188.513183,11.434417,2.372683
2022-09-02,63.031001,15.753633,156.682143,148.385774,148.238470,129.956050,8.911039,11.183483,3.819933,27.874033,...,25.631900,41.841242,352.323766,94.836517,41.422167,45.962883,106.971866,184.660900,11.152533,2.312300
2022-09-09,62.060782,15.613767,156.199979,148.365709,148.275129,128.103050,8.955337,10.929683,3.827000,27.488700,...,25.488633,40.958708,347.989766,93.494983,40.563033,45.595483,105.089067,181.627567,10.902733,2.269500


In [73]:
Adj_Close_Weekly.apply(lambda c: c.rolling(30).mean())

,AA,AAL,AAPL,ABBV,ABC,ABNB,ABST,ACCD,ACHR,ACIW,...,YOU,Z,ZBRA,ZEN,ZG,ZI,ZM,ZS,ZUO,ZY
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-26,63.713106,15.897267,157.236590,148.409657,148.000839,131.545334,8.837461,11.525950,3.805900,28.266600,...,25.716650,42.315583,358.422883,95.823800,41.850450,46.359233,109.077383,188.513183,11.434417,2.372683
2022-09-02,63.031001,15.753633,156.682143,148.385774,148.238470,129.956050,8.911039,11.183483,3.819933,27.874033,...,25.631900,41.841242,352.323766,94.836517,41.422167,45.962883,106.971866,184.660900,11.152533,2.312300
2022-09-09,62.060782,15.613767,156.199979,148.365709,148.275129,128.103050,8.955337,10.929683,3.827000,27.488700,...,25.488633,40.958708,347.989766,93.494983,40.563033,45.595483,105.089067,181.627567,10.902733,2.269500


In [58]:
Adj_Close['Date'] = pd.to_datetime(Adj_Close.index) - pd.to_timedelta(7, unit='d')
Adj_Close_Weekly = Adj_Close.groupby([pd.Grouper(key='Date', freq='W-FRI')]).mean()

,AA,AAL,AAPL,ABBV,ABC,ABNB,ABST,ACCD,ACHR,ACIW,...,YOU,Z,ZBRA,ZEN,ZG,ZI,ZM,ZS,ZUO,ZY
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,12.132500,13.016,113.960338,79.803729,93.855862,NaN,11.860460,39.550999,NaN,28.976000,...,NaN,105.982001,275.529999,107.745999,106.212000,43.692000,483.037994,147.623999,10.3260,NaN
2020-10-09,12.557505,12.438,119.837398,80.041278,95.503824,NaN,12.698096,42.570000,NaN,30.954000,...,NaN,100.810001,288.012000,112.214000,101.232001,42.310000,522.996002,154.095999,10.9860,NaN
2020-10-16,13.083796,12.772,114.843674,77.953114,95.538797,NaN,12.463405,40.130000,NaN,31.409999,...,NaN,97.694000,296.577997,110.558000,98.181999,44.117999,530.122003,148.890002,10.7660,NaN
2020-10-23,12.722343,11.284,112.053352,76.615233,93.743147,NaN,11.247975,35.798001,NaN,30.070000,...,NaN,90.972002,289.038007,107.448001,91.588000,40.630000,504.675995,144.001999,10.1780,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-26,50.620000,13.150,158.256003,136.090002,146.960001,113.252000,11.140000,10.286000,3.498,23.464000,...,23.2080,33.795999,303.947998,76.759999,33.806001,44.227999,80.950000,153.868002,7.8980,2.3160
2022-09-02,49.262500,13.815,155.580002,139.560001,147.167500,117.957500,10.862500,10.720000,3.485,22.145000,...,25.0275,35.993750,297.912491,76.617498,36.157500,42.987499,80.972500,158.747501,7.9975,2.4325
2022-09-09,46.608000,14.040,155.129996,141.378000,141.832001,123.384001,10.742000,12.474000,3.200,22.566000,...,24.6600,35.672000,297.208002,76.729999,35.939999,44.216001,80.284000,180.656003,8.1800,2.7320


In [4]:
import pandas as pd
df = pd.read_csv('Stock Screener\SAVE\Stocks Table.csv')
df.columns
df['Sector'].unique()
df['Industry'].unique()

Index(['Ticker', 'Price', '52-Week-Low', '52-Week-High', '20-MA', '50-MA',
       '100-MA', '150-MA', '200-MA', 'Sector', 'Industry', 'P/E', 'PEG',
       'P/FCF', 'Sales Q/Q %', 'EPS this Y %', 'EPS next Y %', 'Death MACD',
       'Golden MACD', 'MTUM-MA-25', 'MTUM-EMA-25', 'Weekly Death MACD',
       'Weekly Golden MACD'],
      dtype='object')